In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

# Load MoveNet
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']


/Users/jeffrey/Documents/Coding/github/berkeley-ai-hackathon-2025/.venv/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
def detect_pose(frame):
    input_image = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 256, 256)
    input_image = tf.cast(input_image, dtype=tf.int32)

    # Run model
    results = movenet(input_image)
    keypoints = results['output_0'].numpy()[0, 0, :, :2]  # shape: (17, 2)

    return keypoints


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer

# MLP Model
mlp_model = Sequential([
    InputLayer(input_shape=(34,)),  # 17 keypoints * (x, y)
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output: collapse score between 0 and 1
])
mlp_model.compile(optimizer='adam', loss='mse')


/Users/jeffrey/Documents/Coding/github/berkeley-ai-hackathon-2025/.venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [4]:
def preprocess_keypoints(keypoints):
    # Flatten and normalize coordinates to [0, 1]
    flattened = keypoints.flatten()
    return flattened


In [10]:
import time

cap = cv2.VideoCapture(0)  # Or use a file path
time.sleep(2)  # Allow camera to warm up
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for MoveNet
    input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    keypoints = detect_pose(input_frame)
    input_vec = preprocess_keypoints(keypoints)

    # Predict collapse score
    collapse_score = mlp_model.predict(np.expand_dims(input_vec, axis=0))[0][0]

    # Display
    cv2.putText(frame, f'Collapse Score: {collapse_score:.2f}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Pose + Collapse Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Keypoint Capture

In [5]:
# Define keypoint connections (from COCO format)
SKELETON = [
    (0, 1), (1, 3), (0, 2), (2, 4),       # Head -> Shoulders -> Arms
    (5, 7), (7, 9), (6, 8), (8, 10),      # Arms
    (5, 6), (5, 11), (6, 12),             # Torso
    (11, 12), (11, 13), (13, 15),         # Legs
    (12, 14), (14, 16)
]

def draw_pose(frame, keypoints):
    h, w, _ = frame.shape
    # Convert normalized coordinates to pixel
    keypoints_px = [(int(x * w), int(y * h)) for y, x in keypoints]

    # Draw bones
    for start, end in SKELETON:
        x1, y1 = keypoints_px[start]
        x2, y2 = keypoints_px[end]
        cv2.line(frame, (x1, y1), (x2, y2), color=(0, 255, 255), thickness=2)

    # Draw joints
    for (x, y) in keypoints_px:
        cv2.circle(frame, (x, y), radius=5, color=(0, 0, 255), thickness=-1)


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
import os

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

import imageio
from IPython.display import HTML, display

# Load MoveNet Thunder model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# Output file setup
output_file = "pose_labelsTieShoe.csv"
if not os.path.exists(output_file):
    df = pd.DataFrame(columns=[f'kpt_{i}' for i in range(34)] + ['label'])  # 17 keypoints * (x, y) + label
    df.to_csv(output_file, index=False)

def detect_pose(frame):
    input_image = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 256, 256)
    input_image = tf.cast(input_image, dtype=tf.int32)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0, :, :2]  # shape: (17, 2)
    return keypoints

def preprocess_keypoints(keypoints):
    return keypoints.flatten()  # shape: (34,)

def append_to_csv(keypoints, label, file_path):
    flattened = preprocess_keypoints(keypoints)
    row = np.append(flattened, label)
    df = pd.DataFrame([row])
    df.to_csv(file_path, mode='a', header=False, index=False)



In [ ]:
cap = cv2.VideoCapture(0)
print("Press 'z' to save as GOOD pose, 'x' as COLLAPSED pose, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    keypoints = detect_pose(rgb_frame)

    # Draw keypoints on frame
    # for x, y in keypoints:
    #     h, w, _ = frame.shape
    #     cx, cy = int(x * w), int(y * h)
    #     cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
    draw_pose(frame, keypoints)

    cv2.imshow("MoveNet Pose Capture", frame)

    key = cv2.waitKey(10) & 0xFF
    if key == ord('z'):
        print("Saved GOOD pose.")
        append_to_csv(keypoints, 0, output_file)
    elif key == ord('x'):
        print("Saved COLLAPSED pose.")
        append_to_csv(keypoints, 1, output_file)
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Train

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
import os

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

import imageio
from IPython.display import HTML, display

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the CSV
df0 = pd.read_csv("pose_labels.csv")

df1 = pd.read_csv("pose_labelsTieShoe.csv")


# df = pd.concat([df0, df1], axis=0)

df = df0

# Split features and label
X = df.drop("label", axis=1).values  # shape: (N, 34)
y = df["label"].values               # shape: (N,)


In [9]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalize features (0 mean, 1 std)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Define MLP model
model = Sequential([
    Input(shape=(34,)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output: collapse score ∈ [0, 1]
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))

# Save the model
model.save('collapseModel.h5')




Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.6250 - loss: 0.6978 - val_accuracy: 0.6250 - val_loss: 0.7376
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5833 - loss: 0.6223 - val_accuracy: 0.6250 - val_loss: 0.6773
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6042 - loss: 0.6608 - val_accuracy: 0.6250 - val_loss: 0.6188
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5833 - loss: 0.5978 - val_accuracy: 0.6250 - val_loss: 0.5632
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7292 - loss: 0.5426 - val_accuracy: 0.8750 - val_loss: 0.5127
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7292 - loss: 0.5151 - val_accuracy: 1.0000 - val_loss: 0.4657
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9375 - loss: 0.3934 - val_accuracy: 1.0000 - val_loss: 0.4254
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9375 - loss: 0.3758 - val_accuracy: 1.0000 - val_loss: 0.3904

In [11]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0748
Test accuracy: 1.00


In [12]:
# After detecting keypoints:
input_vec = preprocess_keypoints(keypoints).reshape(1, -1)
input_vec = scaler.transform(input_vec)  # Use the same scaler

collapse_score = model.predict(input_vec)[0][0]  # Output: 0-1

print(f"Predicted collapse score: {collapse_score:.2f}")

NameError: name 'keypoints' is not defined

In [12]:
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB for MoveNet
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    keypoints = detect_pose(rgb_frame)
    input_vec = preprocess_keypoints(keypoints).reshape(1, -1)
    input_vec = scaler.transform(input_vec)  # Use the same scaler as training

    # Predict collapse score
    collapse_score = model.predict(input_vec)[0][0]

    # Draw pose
    draw_pose(frame, keypoints)

    # Display collapse score
    cv2.putText(frame, f'Collapse Score: {collapse_score:.2f}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Live Pose + Collapse Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' to quit.


OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!
